# 🚆Train
지하철 데이터 전처리

### TODO

1. 지하철 정류장 데이터 
   - [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 확인
     - [X] 위경도 결측값 분석
     - [X] 결측값 채워진 데이터셋 구하기
     - [X] 노선명 통일
     - [X] 역사명 통일
     - [ ] 두 데이터셋 하나로 결합
     - [ ] 각 노선별 검토 
   - [ ] 서울 및 각 역 위치 지도에 시각화
   - [ ] 이상값 확인
   - [ ] 수도권 바깥 데이터 삭제 
   - [ ] 필요없는 로우 및 컬럼 삭제 
   - [ ] 환승역, 종점, 분기역 정보 가공
2. 지하철 시간표 데이터
   - [ ] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 및 이상값 확인
   - [ ] 방향별 종점별 막차 시간표 정보로 가공

### 0. import

In [699]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import trange, notebook

### 1. 지하철 정류장 데이터

##### 1-a. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [700]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사도로명주소', '역사전화번호','데이터기준일자'])

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도
0,3110,계양,S2801,인천지하철 1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319
1,3111,귤현,S2801,인천지하철 1호선,일반역,-,-,37.566362,126.742498
2,3112,박촌,S2801,인천지하철 1호선,일반역,-,-,37.553525,126.744946
3,3113,임학,S2801,인천지하철 1호선,일반역,-,-,37.545058,126.738642
4,3114,계산,S2801,인천지하철 1호선,일반역,-,-,37.543243,126.728436
...,...,...,...,...,...,...,...,...,...
748,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218
749,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977
750,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151
751,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303


##### 1-b. 결측값 확인

1-b-i. 위경도 결측값 분석

In [701]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
인천지하철 1호선         30
인천지하철 2호선         27
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [702]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station.isnull().sum()

역번호         0
역사명         0
노선번호        0
노선명         0
환승역구분       0
환승노선번호    476
환승노선명     476
역위도       352
역경도       352
dtype: int64

In [703]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64

1-b-ii. 결측값 채워진 데이터셋 구하기

In [704]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 join 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv', encoding='euc_kr')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)

# 필요없는 노선 제거
station_seoul = station_seoul[(station_seoul['노선명'] != '장항선')]

station_seoul

,역번호,역사명,노선명,경도,위도
0,9996,미사,5호선,127.193877,37.560927
1,9995,강일,5호선,127.175930,37.557490
2,4929,김포공항,김포골드라인,126.801868,37.562360
3,4928,고촌,김포골드라인,126.770345,37.601243
4,4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...,...
760,154,종로5가,1호선,127.001849,37.570926
761,153,종로3가,1호선,126.991847,37.570406
762,152,종각,1호선,126.982923,37.570161
763,151,시청,1호선,126.977088,37.565715


In [705]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

1호선          10
2호선          50
3호선          34
4호선          26
5호선          58
6호선          39
7호선          53
7호선(인천)       9
8호선          18
9호선          25
9호선(연장)      13
경강선          11
경부선          39
경원선          30
경의중앙선        31
경인선          20
경춘선          19
공항철도1호선      14
과천선           9
김포골드라인       10
분당선          35
서해선          12
수인선          18
신림선          11
신분당선          6
신분당선(연장)      7
신분당선(연장2)     3
안산선          13
에버라인선        15
우이신설선        13
의정부선         15
인천1호선        30
인천2호선        27
일산선          11
중앙선          21
진접선           3
Name: 노선명, dtype: int64

In [706]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station_seoul.isnull().sum()

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

역번호    0
역사명    0
노선명    0
경도     0
위도     0
dtype: int64

1-b-iii. 노선명 통일

In [707]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

{'안산과천선', '인천지하철 1호선', '도시철도 7호선', '에버라인', '인천국제공항선', '수도권 경량도시철도 신림선', '인천지하철 2호선', '의정부', '수도권  도시철도 9호선'}
{'인천2호선', '의정부선', '공항철도1호선', '중앙선', '과천선', '9호선(연장)', '에버라인선', '인천1호선', '안산선', '신림선', '신분당선(연장2)', '7호선(인천)', '신분당선(연장)', '9호선'}


In [708]:
# 노선명 통일
station["노선명"] = station["노선명"].replace({
    "도시철도 7호선": "7호선",
    "수도권  도시철도 9호선": "9호선",
    "수도권 경량도시철도 신림선": "신림선",
    "인천지하철 1호선": "인천1호선",
    "인천지하철 2호선": "인천2호선",
    "인천국제공항선": "공항철도1호선",
    "에버라인": "에버라인선",
    "의정부": "의정부선"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"])
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 안산과천선의 경우, 역사명 통일한 이후 안산선 / 과천선으로 나눌 예정

{'안산과천선'}
{'중앙선', '과천선', '9호선(연장)', '안산선', '7호선(인천)', '신분당선(연장2)', '신분당선(연장)'}


1-b-iv. 역사명 통일

In [709]:
# 기존 데이터셋과 역번호 컬럼 데이터 형식 일치시키기
station_seoul = station_seoul.astype({'역번호': 'str'})

# 컬럼 데이터 형식 확인
print(station.dtypes)
print(station_seoul.dtypes)

역번호        object
역사명        object
노선번호       object
노선명        object
환승역구분      object
환승노선번호     object
환승노선명      object
위도        float64
경도        float64
dtype: object
역번호     object
역사명     object
노선명     object
경도     float64
위도     float64
dtype: object


In [710]:
# 둘 중 한 데이터셋에만 있는 역번호, 역사명 확인
station_num = set(station["역번호"])
station_num_seoul = set(station_seoul["역번호"])
sub_station_num = sorted(station_num - station_num_seoul)
sub_station_num_seoul = sorted(station_num_seoul - station_num)

print(sub_station_num)
print(sub_station_num_seoul)

station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)


['0124', '0125', '0126', '0127', '0128', '0129', '0130', '0131', '0132', '0133', '0201', '0202', '0203', '0204', '0205', '0206', '0207', '0208', '0209', '0210', '0211', '0212', '0213', '0214', '0215', '0216', '0217', '0218', '0219', '0220', '0221', '0222', '0223', '0224', '0225', '0226', '0227', '0228', '0229', '0230', '0231', '0232', '0233', '0234', '0235', '0236', '0237', '0238', '0239', '0240', '0241', '0242', '0243', '0319', '0320', '0321', '0322', '0323', '0324', '0325', '0326', '0327', '0328', '0329', '0330', '0331', '0332', '0333', '0334', '0335', '0336', '0337', '0338', '0339', '0340', '0341', '0342', '0343', '0344', '0345', '0346', '0347', '0348', '0349', '0350', '0351', '0352', '0405', '0406', '0408', '0409', '0410', '0411', '0412', '0413', '0414', '0415', '0416', '0417', '0418', '0419', '0420', '0421', '0422', '0423', '0424', '0425', '0426', '0427', '0428', '0429', '0430', '0431', '0432', '0433', '0434', '0510', '0511', '0512', '0513', '0514', '0515', '0516', '0517', '0518',

In [711]:
# 역이름 통일
# 역이름 맨 끝에 등장하는'역'자 제거
station['역사명'] = station['역사명'].str.replace(pat=r'역$', repl=r'', regex=True)
# 서울역은 다시 '역' 추가 처리 
station['역사명'] = station['역사명'].str.replace(pat=r'^서울$', repl=r'서울역', regex=True)

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['검단오류', '관악산', '교대(법원·검찰청)', '기흥(백남준아트센터)', '낙성대(강감찬)', '남한산성입구(성남법원·검찰청)', '동대문역사문화공원(DDP)', '동백(용인세브란스)', '동작', '배방', '봉명', '상봉', '서울역(경의선)', '성신여대입구', '수원(분당)', '시민공원', '시우', '시청·용인대', '신창(순천향대)', '신촌(지하)', '쌍용(나사렛대)', '아산', '아시아드경기장', '온수', '온양온천', '왕십리', '용마산(용마폭포공원)', '운동장·송담대(중앙시장)', '운연', '이촌', '전대·에버랜드', '종로3가(탑골공원)', '청량리', '탕정', '하남시청(덕풍∙신장)', '흑석']
['검단오류(검단산업단지)', '관악산(서울대)', '교대(법원.검찰청)', '낙성대', '남한산성입구(성남법원.검찰청)', '동대문역사문화공원', '동백', '둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '시민공원(문화창작지대)', '시청.용인대', '아시아드경기장(공촌사거리)', '언주', '용마산', '운동장.송담대', '운연(서창)', '원곡', '이수', '전대.에버랜드', '중앙보훈병원', '하남시청(덕풍-신장)', '한성백제', '흑석(중앙대입구)']


In [712]:
# 특수문자 .으로 통일
station['역사명'] = station['역사명'].str.replace(pat=r'[·∙-]', repl=r'.', regex=True)
station_seoul['역사명'] = station_seoul['역사명'].str.replace(
    pat=r'[·∙-]', repl=r'.', regex=True)

# 수동 맞추기 -> 괄호 생기는 방향으로 (단, 지하/지상 정보, 노선명인 경우 제거)
station["역사명"] = station["역사명"].replace({
    "동작": "동작(현충원)",
    "검단오류": "검단오류(검단산업단지)",
    "관악산": "관악산(서울대)",
    "서울역(경의선)": "서울역",
    "시민공원": "시민공원(문화창작지대)",
    "수원(분당)": "수원",
    "신촌(지하)": "신촌",
    "운연": "운연(서창)",
    "흑석": "흑석(중앙대입구)",
    "아시아드경기장": "아시아드경기장(공촌사거리)"
})

station_seoul["역사명"] = station_seoul["역사명"].replace({
    "낙성대": "낙성대(강감찬)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "동백": "동백(용인세브란스)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "용마산": "용마산(용마폭포공원)"
})

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)


['기흥(백남준아트센터)', '배방', '봉명', '상봉', '성신여대입구', '시우', '신창(순천향대)', '쌍용(나사렛대)', '아산', '온수', '온양온천', '왕십리', '이촌', '종로3가(탑골공원)', '청량리', '탕정']
['둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '언주', '원곡', '이수', '중앙보훈병원', '한성백제']


In [713]:
# 안산과천선 -> 안산선 / 과천선
station_name_ansan = set(station_seoul['역사명'][station_seoul['노선명'] == '안산선']) 
station_name_gwacheon = set(
    station_seoul['역사명'][station_seoul['노선명'] == '과천선'])

station.loc[(station['노선명'] == '안산과천선') & (station['역사명'].isin(station_name_ansan)), "노선명"] = "안산선"
station.loc[(station['노선명'] == '안산과천선') & (station['역사명'].isin(station_name_gwacheon)), "노선명"] = "과천선"

# 잘 바뀌었는지 확인
station["노선명"].value_counts().sort_index()

1호선        10
2호선        50
3호선        34
4호선        26
5호선        56
6호선        39
7호선        53
8호선        18
9호선        25
경강선        11
경부선        45
경원선        24
경의중앙선      57
경인선        20
경춘선        24
공항철도1호선    14
과천선         8
김포골드라인     10
분당선        37
서해선        12
수인선        26
신림선        11
신분당선       16
안산선        13
에버라인선      15
우이신설선      13
의정부선       15
인천1호선      30
인천2호선      27
일산선        11
진접선         3
Name: 노선명, dtype: int64